In [1]:
#pragma cling add_include_path("/home/simon/Projects/liblsd/include")

In [2]:
#pragma cling add_library_path("/home/simon/Projects/liblsd/lib")
#pragma cling load("liblsd.so")

In [3]:
#include <lsd.h>

In [4]:
Pr* opt = new Pr();

In [5]:
// tree file
opt->inFile="./Dengue.phy_phyml_tree.txt";
// date file
opt->inDateFile="./Dengue.date";
// sequence length
opt->seqLength=1000;
// partition file
opt->partitionFile = "";
// outfile
opt->outFile = "./Dengue.lsd";
opt->treeFile1=opt->outFile+".nexus";
opt->treeFile2=opt->outFile+".newick";
opt->treeFile3=opt->outFile+".date.newick";
// number of trees
opt->nbData=1;
// outgroup file
opt->fnOutgroup = "";
if (opt->fnOutgroup!=""){
    opt->keepOutgroup=true;
    opt->estimate_root="k";
}
// estimate root
opt->estimate_root = "a";
// variance
opt->variance = 2;
// use temporal constraints
opt->constraint = true;
// parameter of variance
opt->c = 10;
// lower bound for rate
opt->rho_min = 1.0000000e-10;
// filename for given rates per tree
opt->rate = "";
if(opt->rate!=""){
    opt->givenRate = true;
}
// root date
opt->mrca=0.00000;
// -z: tips date
opt->leaves=1.00000;
// -f: samples for bootstrap
opt->nbSampling=0;
if(opt->nbSampling>0){
    opt->ci=true;
    bool fflag = true;
}

In [6]:
checkRooted(opt);

(void) @0x7ff3a3905da0


In [7]:
opt->rooted;

(bool) false


In [8]:
opt->nbBranches;

(int) 32


In [9]:
printInterface( stdout, opt );


LEAST-SQUARE METHODS TO ESTIMATE RATES AND DATES - v0.3beta 


Input files:
  I                                 Input tree file : ./Dengue.phy_phyml_tree.txt
  D                                 Input date file : ./Dengue.date
  P                                  Partition file : No
Output file:
  O                                    Output file  : ./Dengue.lsd
Parameters:
  C                                With constraints : Yes
  T                        Lower bound for the rate : 1.000e-10
  V                                  With variances : Yes, use variances based on branches estimated by LSD
  B                          Parameter of variances : 10
  S                                 Sequence Length : 1000
  R                               Estimate the root : Use fast method to search on all branches
  W                         Given substitution rate : No
  G                                 Given outgroups : No
  N                               Multiple data set : No
  F        

(void) @0x7ff3a3905da0


In [10]:
FILE * tree = fopen(opt->inFile.c_str(),"rt");

In [11]:
FILE * tree1 = fopen(opt->treeFile1.c_str(),"wt");
FILE * tree2 = fopen(opt->treeFile2.c_str(),"wt");
FILE * tree3 = fopen(opt->treeFile3.c_str(),"wt");
FILE * gr=NULL; //given rate

In [12]:
if (tree1==NULL || tree2==NULL || tree3==NULL){
    cout<<"Error: can not create the output tree file "<<opt->treeFile1<<endl;
};

In [13]:
fprintf(tree1,"#NEXUS\n");
if (opt->givenRate) {
    gr = fopen(opt->rate.c_str(),"rt");
}

In [14]:
if (opt->partitionFile!="") {
    cout<<"Reading partition file"<<endl;
    readPartitionFile(opt);
}

In [15]:
bool constraintConsistent=true;
int s=0;
Node** nodes = new Node*[opt->nbBranches+1];

In [16]:
FILE * result = fopen(opt->outFile.c_str(),"wt");
if (result==NULL){
    cout<<"Error: can not create the output file "<<opt->outFile<<endl;
}

In [17]:
opt->initConstraints();

(void) nullptr


In [18]:
s=tree2data(tree,opt,nodes,constraintConsistent);

(int) 2


In [19]:
nodes = unrooted2rooted(opt,nodes,s);

(Node **) 0x7ff39e2d1e70


In [20]:
computeVariance(opt,nodes);
constraintConsistent=initConstraint(opt, nodes);

(bool) true


In [21]:
int r;
r=estimate_root_with_constraint_rooted(opt,nodes);

Optimizing the root position on the branch 1 ... 42.0091519071
Optimizing the root position on the branch 2 ... 42.0091519071
Optimizing the root position on the branch 3 ... 16.4409450364
Optimizing the root position on the branch 4 ... 10.7816976391
Optimizing the root position on the branch 5 ... 10.2964176009
Optimizing the root position on the branch 6 ... 91.0173964239
Optimizing the root position on the branch 7 ... 96.3237700264
Optimizing the root position on the branch 8 ... 101.6223452091
Optimizing the root position on the branch 9 ... 104.0977559181
Optimizing the root position on the branch 10 ... 104.6322606201
Optimizing the root position on the branch 11 ... 105.1553302752
Optimizing the root position on the branch 12 ... 110.1395067371
Optimizing the root position on the branch 13 ... 111.5631693268
Optimizing the root position on the branch 14 ... 16.4409450364
Optimizing the root position on the branch 15 ... 38.2170064318
Optimizing the root position on the branch 

(int) 26


In [22]:
//int y = 1;
double br = 0.0;
double TMRCA = 0.0;
double objective = 0.0;
if (r>0){
    Node** nodes_new = cloneLeaves(opt,nodes,0);
    vector<int>::iterator iter=nodes[0]->suc.begin();
    int s1=(*iter);
    iter++;
    int s2=(*iter);
    for (int i=opt->nbINodes; i<=opt->nbBranches; i++) {
      nodes_new[i]->status=nodes[i]->status;
    }
    reroot_rootedtree(br,r,s1,s2,opt,nodes,nodes_new);
    with_constraint_active_set_lambda_multirates(br,opt,nodes_new,true);
    // output(br,y,opt,nodes_new,result,tree1,tree2,tree3);
    TMRCA = nodes_new[0]->D;
    objective = opt->objective;
    cout << newickDate(0,opt,nodes_new).c_str() << endl;
    for (int i=0;i<opt->nbBranches+1;i++) delete nodes_new[i];
    delete[] nodes_new;
}

(Thai63:39.7913,((((Thai84:11.7871,Thai78:5.78714):14.1397,SLanka78:19.9268):18.1279,((Philip64:3.92898,Philip84:23.929):6.91755,Philip56:2.84653):13.2082):5.90161,(((((((((ElSal83:1.33024,Brazi82:0.330244):0,Mexico84:2.33024):0,NewCal84:2.33024):1.03942,ElSal94:13.3697):1.55312,PRico86:6.92278):1.48234,Tahiti79:1.40512):0,Tahiti85:7.40512):4.0766,Indon77:3.48172):0.090927,Indon76:2.57264):39.3837):10.8349);



In [23]:
fclose(tree);
fclose(result);
fclose(tree1);
fclose(tree2);
fclose(tree3);
if (opt->rate!="") fclose(gr);

In [24]:
for (int i=0;i<opt->nbBranches+1;i++) delete nodes[i];
delete[] nodes;
delete opt;

(void) @0x7ff3a3905da0
